# Exercise answers

# Table of Contents

- [Exercise 1](#Exercise-1)
- [Exercise 2](#Exercise-2)
- [Exercise_3](#Exercise-3)
- [Exercise 4](#Exercise-4)
- [More questions](#More-questions)

# Exercise 1

- Back to the [Table of Contents](#Table-of-Contents)

Use your database client of choice to interact with the database to answer the questions that follow.

Questions:

- 1) familiarize yourself with the column names and what they represent. Select 100 rows of the hh_indcase_spells table. What is the data in the last row? Do any of the columns have unexpected or confusing data?

        SELECT *
        FROM idhs.hh_indcase_spells
        LIMIT 100;

- 2) instead of selecting all columns, select only the columns corresponding to ssn (hashed), start date, sex, race and benefit type. Limit your query to 200. What is the data in the last row?

        SELECT ssn_hash, start_date, sex, rac, rootrace, benefit_type
        FROM idhs.hh_indcase_spells
        LIMIT 200;
        
- 3) Browse other tables and practice selecting specific columns in each table (hint: first select a few rows of all the columns in a table to see what the column names and contents look like, then choose columns that interest you to focus on, perhaps by getting more rows so you can see more of the data).

        SELECT *
        FROM ides.il_wage
        LIMIT 200;

# Exercise 2

- Back to the [Table of Contents](#Table-of-Contents)

Use your database client of choice to interact with the database to answer the questions that follow.

For each question, enter:

- The SQL query you used to find the answer.
- The answer to the question.

Questions:

- 4) Using the idhs.hh_indcase_spells table. Find how many individuals received welfare in the first quarter of 2015.

        SELECT COUNT( DISTINCT( ssn_hash ) ) AS individual_count
        FROM idhs.hh_indcase_spells
        WHERE ( start_date >= '2015-01-01' ) AND ( end_date <= '2015-03-31' );

- 5) Use the same table. Count how many distinct welfare cases there were in 2015.

        SELECT COUNT( DISTINCT( ch_dpa_caseid ) ) AS individual_count
        FROM idhs.hh_indcase_spells
        WHERE ( start_date >= '2015-01-01' ) AND ( end_date <= '2015-12-31' );

- Extra credit - design and implement a WHERE clause that better implements "in 2015".:

        SELECT COUNT( DISTINCT( ssn_hash ) ) AS individual_count
        FROM idhs.hh_indcase_spells
        WHERE ( ( ( start_date >= '2015-01-01' ) AND ( start_date <= '2015-03-31' )  )
            OR ( ( start_date < '2015-01-01' ) AND ( end_date > '2015-12-31' ) )
            OR ( ( end_date>= '2015-01-01' ) AND ( end_date <= '2015-12-31' ) ) );

# Exercise 3

- Back to the [Table of Contents](#Table-of-Contents)

Use your database client of choice to interact with the database to answer the questions that follow.

For each question, enter:

- The SQL query you used to find the answer.
- The answer to the question.

Questions:

- 6) What is the percentage of welfare recipients in each racial category in year 2015?

        SELECT ( COUNT( DISTINCT( ssn_hash ) ) / ( 155212 * 1.0 ) ) AS individual_percentage, rac
        FROM idhs.hh_indcase_spells
        WHERE start_date >= '2015-01-01' AND end_date <= '2015-12-31'
        GROUP BY rac;

        SELECT ( COUNT( DISTINCT( ssn_hash ) ) / ( 155212 * 1.0 ) ) AS individual_percentage, rootrace
        FROM idhs.hh_indcase_spells
        WHERE start_date >= '2015-01-01' AND end_date <= '2015-12-31'
        GROUP BY rootrace;

- 7) What is the average duration of a spell in 2015?

        SELECT AVG( AGE( end_date, start_date ) ) AS average_spell
        FROM idhs.hh_indcase_spells
        WHERE start_date >= '2015-01-01' AND end_date <= '2015-12-31';
        
- 8) Which type of benefit did welfare recipients receive the most in 2015?

        SELECT COUNT( * ) as spell_count, benefit_type
        FROM idhs.hh_indcase_spells
        GROUP BY benefit_type
        ORDER BY spell_count DESC;

# Exercise 4

- Back to the [Table of Contents](#Table-of-Contents)

Reproduce the above earnings investigation starting with persons who have corrections records, rather than welfare recipients.

## Question: What kinds of jobs do former prisoners have, and for what employers?

First attempt to tie corrections persons to wage records to look at how former prisoners are employed.  To start, find wage records that match ildoc.person records based on having the same SSN hash:

    /* Lists earnings of 10 matching records */
    SELECT p.id AS person_id, p.ildoc_docnbr AS docnbr, iw.wage, iw.year, iw.quarter
    FROM ildoc.person p
    JOIN ides.il_wage iw
    ON p.ssn_hash = iw.ssn
    LIMIT 10;

Use regular WHERE clause to further filter:

    /* Lists earnings of matching records where individual is female. */
    SELECT p.id AS person_id, p.ildoc_docnbr AS docnbr, iw.wage, iw.year, iw.quarter
    FROM ildoc.person p
    JOIN ides.il_wage iw
    ON p.ssn_hash = iw.ssn
    WHERE p.sex = 'F'
    LIMIT 10;
    
For this type of simple `INNER JOIN`, you can also omit the "`JOIN`" in the `FROM` clause and put the `JOIN` condition into the `WHERE` clause:

    /* Lists earnings of matching records where individual is female. */
    SELECT p.id AS person_id, p.ildoc_docnbr AS docnbr, iw.wage, iw.year, iw.quarter
    FROM ildoc.person p, ides.il_wage iw
    WHERE p.ssn_hash = iw.ssn
        AND p.sex = 'F'
    LIMIT 10;

## join corrections, wage and employer data

Add the employer information to the previous table only if the employer number in the wage table and the employer table match. So an INNER JOIN is used.

The complete query:

    SELECT p.id AS person_id, p.ildoc_docnbr AS docnbr, iw.wage, iw.year, iw.quarter
    FROM ildoc.person p
    LEFT JOIN ides.il_wage iw
        ON p.ssn_hash = iw.ssn
    INNER JOIN ides.il_qcew_employers iqe
        ON iw.empr_no = iqe.empr_no AND iw.seinunit = iqe.seinunit AND iw.ein = iqe.ein
    LIMIT 10;